In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import os

import warnings 
warnings.filterwarnings('ignore')

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

# Cálculo del St

Calculamos el tiempo hasta que un dispositivo vuelva a aparecer en una subasta RTB como la fecha de la primera aparición en la ventana menos la fecha de inicio de la ventana.

In [4]:
training_set_subastas = auctions
training_set_subastas.sort_values(ascending=True,by='date',inplace=True)

In [5]:
#Definimos la fecha de inicio de cada ventana
fecha_iv1 = pd.to_datetime('2019-04-18')
fecha_iv2 = pd.to_datetime('2019-04-19')
fecha_iv3 = pd.to_datetime('2019-04-20')
fecha_iv4 = pd.to_datetime('2019-04-21')
fecha_iv5 = pd.to_datetime('2019-04-22')
fecha_iv6 = pd.to_datetime('2019-04-23')
fecha_iv7 = pd.to_datetime('2019-04-24')

fecha_inicio_ventana = [fecha_iv1,fecha_iv2,fecha_iv3,fecha_iv4,fecha_iv5,fecha_iv6,fecha_iv7]

In [6]:
#Separamos el set de datos de subastas en "ventanas" de 3 días cada una.
ventanas_subastas = []
for i in range(0,7):
    ventanas_subastas.append(training_set_subastas.loc[(training_set_subastas['date'] > fecha_inicio_ventana[i]) & (training_set_subastas['date'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

In [7]:
for i in range(0,7):
    ventanas_subastas[i]['st'] = (ventanas_subastas[i].groupby('device_id')['date'].transform(min) - fecha_inicio_ventana[i]).dt.total_seconds()

In [8]:
labels = []

for i in range(0,7):
    labels.append(ventanas_subastas[i][['device_id','st']].drop_duplicates())

In [9]:
#Veamos la cantidad de registros del label de cada ventana
for i in range(0,7):
    print("Cantidad de registros en el label " + str(i+1) + ": " + str(labels[i].shape[0]))

Cantidad de registros en el label 1: 318325
Cantidad de registros en el label 2: 316149
Cantidad de registros en el label 3: 324622
Cantidad de registros en el label 4: 326945
Cantidad de registros en el label 5: 325310
Cantidad de registros en el label 6: 322403
Cantidad de registros en el label 7: 329779


In [10]:
#Veamos la forma que tiene un label
labels[0].head()

,device_id,st
10129864,1826643666390887030,0.015050
25285906,7037174172278258682,0.029014
3946062,3392065368947589877,0.057540
3946063,1228982273563226229,0.126828
24607726,4123059034628125459,0.132510


# Cálculo del Sc

De la misma forma que con St, calculamos el tiempo hasta que un dispositivo vuelva a instalar como la fecha de la primera aparición en la ventana menos la fecha de inicio de la ventana.

In [11]:
training_set_instalaciones = installs
training_set_instalaciones.sort_values(ascending=True,by='created',inplace=True)

In [12]:
#Separamos el set de datos de instalaciones en "ventanas" de 3 días cada una.
ventanas_instalaciones = []
for i in range(0,7):
    ventanas_instalaciones.append(training_set_instalaciones.loc[(training_set_instalaciones['created'] > fecha_inicio_ventana[i]) & (training_set_instalaciones['created'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

In [13]:
for i in range(0,7):
    ventanas_instalaciones[i]['sc'] = (ventanas_instalaciones[i].groupby('ref_hash')['created'].transform(min) - fecha_inicio_ventana[i]).dt.total_seconds()

In [14]:
tiempo_maximo = 259200 #tiempo de 3 días en segundos

for i in range(0,7):
    instalaciones_a_mergear = ventanas_instalaciones[i][['ref_hash','sc']].drop_duplicates()
    instalaciones_a_mergear.columns = ['device_id', 'sc']
    labels[i] = labels[i].merge(instalaciones_a_mergear,on='device_id',how='outer')
    labels[i].fillna(value=tiempo_maximo,inplace=True)

In [15]:
#Veamos la cantidad de registros del label de cada ventana luego del join
for i in range(0,7):
    print("Cantidad de registros en el label " + str(i+1) + ": " + str(labels[i].shape[0]))

Cantidad de registros en el label 1: 385959
Cantidad de registros en el label 2: 382751
Cantidad de registros en el label 3: 391936
Cantidad de registros en el label 4: 396735
Cantidad de registros en el label 5: 395720
Cantidad de registros en el label 6: 394475
Cantidad de registros en el label 7: 403828


Los label por ventana quedan de la siguiente forma:

In [16]:
labels[0].head()

,device_id,st,sc
0,1826643666390887030,0.015050,259200.000
1,7037174172278258682,0.029014,259200.000
2,3392065368947589877,0.057540,259200.000
3,1228982273563226229,0.126828,259200.000
4,4123059034628125459,0.132510,17800.664


# FEATURES
Creamos features en base a los datasets. Los features van a estar divididos por dispositivo para poder mergearlos en un unico dataframe con features de todos los dataframes.

### Features de Auctions

In [17]:
ventanas_auctions = []
for i in range(0,7):
    ventanas_auctions.append(auctions.loc[(auctions['date'] > fecha_inicio_ventana[i]) & (auctions['date'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

#### Cantidad de subastas por ventana

In [18]:
features_auctions = []

for ventana in ventanas_auctions:
    features_ventana = ventana.groupby('device_id').agg({'device_id':'size'})
    features_ventana.columns = ['cantidad_de_subastas_en_ventana']
    features_auctions.append(features_ventana)

In [19]:
for i in range(0,7):
    features_auctions[i] = features_auctions[i].reset_index()

In [20]:
features_auctions[0].head()

,device_id,cantidad_de_subastas_en_ventana
0,41863526108385,35
1,135153013040192,8
2,161514654074162,6
3,181891380775191,1
4,186034136943920,7


In [21]:
features_auctions[0].shape

(318325, 2)

### Features de Events

In [22]:
ventanas_events = []
for i in range(0,7):
    ventanas_events.append(events.loc[(events['date'] > fecha_inicio_ventana[i]) & (events['date'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

#### Cantidad de eventos por ventana

In [23]:
features_events = []

for ventana in ventanas_events:
    ventana['kind'].fillna(0,inplace=True)
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size',
                                                        'event_id':lambda x: (x.mode())[0],
                                                        'application_id':lambda x: (x.mode())[0],
                                                        'kind':lambda x: (x.mode())[0]})
    
    features_ventana.columns = ['cantidad_de_eventos_en_ventana','evento_mas_frecuente','aplicacion_mas_usada','tipo_evento']
    features_events.append(features_ventana)

#### Versión más tipica de SO registrada en los eventos para cada dispositivo

In [24]:
for index,ventana in enumerate(ventanas_events):
    ventana_temp = ventana
    ventana_temp['device_os_version'] = ventana_temp['device_os_version'].fillna(value=0)
    features_events[index] = features_events[index].merge(ventana_temp.groupby('ref_hash').\
                                              agg({'device_os_version':lambda x: (x.mode())[0]}),
                                      left_on='ref_hash',
                                      right_index=True,
                                      how='outer')

In [25]:
for i in range(0,7):
    features_events[i] = features_events[i].reset_index().rename(columns={'ref_hash':'device_id'})

In [26]:
features_events[0].head()

,device_id,cantidad_de_eventos_en_ventana,evento_mas_frecuente,aplicacion_mas_usada,tipo_evento,device_os_version
0,41863526108385,88,1,210,4.017674e+18,0.000000e+00
1,161514654074162,8,23,121,6.081165e+18,0.000000e+00
2,186034136943920,13,1,210,4.017674e+18,0.000000e+00
3,360710529886978,3,23,14,6.081165e+18,7.391844e+18
4,365882020742330,57,15,302,5.500848e+18,0.000000e+00


In [27]:
features_events[0].shape

(166002, 6)

### Clicks

In [28]:
ventanas_clicks = []
for i in range(0,7):
    ventanas_clicks.append(clicks.loc[(clicks['created'] > fecha_inicio_ventana[i]) & (clicks['created'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

#### Cantidad de clicks por ventana

In [29]:
features_clicks = []

for ventana in ventanas_clicks:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_clicks_en_ventana']
    features_clicks.append(features_ventana)

#### Tiempo promedio en hacer click

In [30]:
# En caso de que sea nulo se llena con tiempo maximo
tiempo_maximo = 259200
for index, ventana in enumerate(ventanas_clicks):
    temp = ventana.groupby('ref_hash').agg({'timeToClick' : 'mean'})
    temp.columns = ['tiempo_de_click_promedio']
    
    features_clicks[index] = features_clicks[index].merge(temp, how='outer', on='ref_hash')
    features_clicks[index]['tiempo_de_click_promedio'].fillna(tiempo_maximo, inplace=True)

#### Region en pantalla del click

Se divide en una cuadricula de 9 sectores.Por un lado: Superior, subdividido en izquierda, centro y derecha; Centro con mismas subdivisiones e Inferior también con mismas subdivisiones.

In [31]:
#Me quedo con la region mas común para cada dispositivo
for index, ventana in enumerate(ventanas_clicks):
    ventana_temp = ventana[['ref_hash','touchX','touchY']].fillna(value=-1).groupby('ref_hash')\
                        .agg({'touchX':lambda x: (x.mode())[0],
                              'touchY':lambda x: (x.mode())[0]}).reset_index()
    ventana_temp['SUP_D'] = 0
    ventana_temp['SUP_C'] = 0
    ventana_temp['SUP_I'] = 0
    ventana_temp['CENT_D'] = 0
    ventana_temp['CENT_C'] = 0
    ventana_temp['CENT_I'] = 0
    ventana_temp['INF_D'] = 0
    ventana_temp['INF_C'] = 0
    ventana_temp['INF_I'] = 0
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=10.66),'SUP_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=10.66),'SUP_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=10.66),'SUP_I'] = 1
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_I'] = 1
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_I'] = 1
    
    features_clicks[index] = pd.merge(features_clicks[index],\
        ventana_temp[['ref_hash','SUP_D','SUP_C','SUP_I','CENT_D','CENT_C','CENT_I','INF_D','INF_C','INF_I']].drop_duplicates(),\
                              on='ref_hash',how='inner')

In [32]:
for i in range(0,7):
    features_clicks[i] = features_clicks[i].rename(columns={'ref_hash':'device_id'})

In [33]:
features_clicks[0].head()

,device_id,cantidad_de_clicks_en_ventana,tiempo_de_click_promedio,SUP_D,SUP_C,SUP_I,CENT_D,CENT_C,CENT_I,INF_D,INF_C,INF_I
0,1461247282174365,1,229.257,0,0,0,0,0,0,1,0,0
1,5470466329076030,1,259200.000,0,0,0,0,0,0,0,0,0
2,5534899846690585,1,10.223,0,0,0,0,0,0,1,0,0
3,7429113196145773,1,259200.000,0,0,0,0,0,0,1,0,0
4,8577232270715133,1,259200.000,0,0,0,0,0,0,0,0,0


In [34]:
features_clicks[0].shape

(9185, 12)

### Installs

In [35]:
ventanas_installs = []
for i in range(0,7):
    ventanas_installs.append(installs.loc[(installs['created'] > fecha_inicio_ventana[i]) & (installs['created'] < (fecha_inicio_ventana[i] + pd.Timedelta('3 days')))])

#### Cantidad de instalaciones por ventana

In [36]:
features_installs = []

for ventana in ventanas_installs:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_instalaciones_en_ventana']
    features_installs.append(features_ventana)

In [37]:
#Las columnas kind y wifi "rompen" el fit transform()
# OneHotEncoder().fit_transform(features_por_ventana[0])

for index, ventana in enumerate(ventanas_installs):
    ventana['Wifi_cat'] = ventana['Wifi_cat'].cat.add_categories([''])
    ventana['kind'].fillna('',inplace=True)
    features_ventana = ventana.groupby('ref_hash').agg({'Wifi_cat':lambda x: (x.mode())[0],
                                                        'kind':lambda x: (x.mode())[0]})
    
    features_ventana.columns = ['wifi','kind']
    
    features_installs[index] = features_installs[index].merge(features_ventana,   
                                                              left_on='ref_hash',
                                                              right_index=True,
                                                              how='outer')
    features_installs[index]['wifi'].fillna('', inplace=True)
    features_installs[index]['kind'].fillna('', inplace=True)

In [38]:
for i in range (0,7):
    ventanas_installs[i] =pd.merge(ventanas_installs[i],
             (clicks.loc[clicks['created']<fecha_inicio_ventana[i]].groupby('ref_hash').size().reset_index(name='cant_clicks_previos')),
             on='ref_hash',
             how='outer')
    ventanas_installs[i]['cant_clicks_previos'].fillna(0,inplace=True)

#### Relación cantidad de instalacions / cantidad de eventos por dispositivo

In [39]:
for i in range(0,7):
    feature_porc = pd.merge(ventanas_events[i].groupby('ref_hash').size().reset_index(name='cantidad_eventos_x_dev'),
                       ventanas_installs[i].groupby('ref_hash').size().reset_index(name='cantidad_inst_x_dev'),
                       on='ref_hash',
                       how='inner')
    feature_porc['Porc_I-E'] =  feature_porc['cantidad_inst_x_dev'] / feature_porc['cantidad_eventos_x_dev']
    feature_porc.set_index('ref_hash',inplace=True)
    feature_porc['Porc_I-E'] = feature_porc['Porc_I-E'].astype('float16')
    del feature_porc['cantidad_inst_x_dev'] 
    del feature_porc['cantidad_eventos_x_dev']
    features_installs[i] = features_installs[i].merge(feature_porc, how='outer', left_index=True, right_index=True)
    features_installs[i].fillna(0, inplace=True)

#### Tipo de instalación open
Notamos que los tipos de instalaciones que más se repiten son las de tipo open. Creamos un feature para determinar si el dispositivo tiene ese tipo de instalación o no.

In [51]:
for i in range(0,7):
    temp = ventanas_installs[i][['ref_hash','kind']]
    temp['instalacion_open'] = ventanas_installs[i]['kind'].str.lower().str.contains('open')
    temp = temp.loc[temp['instalacion_open']== True][['ref_hash','instalacion_open']].sort_values(by='ref_hash').drop_duplicates()
    temp.set_index('ref_hash',inplace=True)
    
    features_installs[i] = features_installs[i].merge(temp,left_index=True,right_index=True, how='outer')
    features_installs[i]['instalacion_open'].fillna(False, inplace=True)

In [52]:
for i in range(0,7):
    features_installs[i] = features_installs[i].reset_index().rename(columns={'ref_hash':'device_id'})

In [53]:
features_installs[0].head()

,device_id,cantidad_de_instalaciones_en_ventana,wifi,kind,Porc_I-E,instalacion_open
0,41863526108385,4.0,Sin Wifi,,0.045441,True
1,135153013040192,1.0,Sin Definir,,0.000000,False
2,186034136943920,1.0,Sin Definir,list view,0.076904,False
3,365882020742330,1.0,Sin Wifi,,0.017548,False
4,519199987760489,1.0,Con Wifi,,0.500000,False


In [54]:
features_installs[0].shape

(134114, 6)

## Mergeo todos los features por dispositivo

In [55]:
features_por_ventana = []

for i in range(0,7):
    features_por_ventana.append(features_auctions[i].merge(features_clicks[i],on='device_id',how='outer'))
    
    features_por_ventana[i] = features_por_ventana[i].merge(features_installs[i],on='device_id',how='outer')
    
    features_por_ventana[i] = features_por_ventana[i].merge(features_events[i],on='device_id',how='outer')
    
    features_por_ventana[i].fillna(0,inplace=True)
    
    features_por_ventana[i] = pd.merge(labels[i][['device_id']],features_por_ventana[i],on='device_id',how='inner')
    
    features_por_ventana[i].sort_values(by='device_id',inplace=True)

In [56]:
features_por_ventana[0].head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_de_clicks_en_ventana,tiempo_de_click_promedio,SUP_D,SUP_C,SUP_I,CENT_D,CENT_C,CENT_I,...,cantidad_de_instalaciones_en_ventana,wifi,kind,Porc_I-E,instalacion_open,cantidad_de_eventos_en_ventana,evento_mas_frecuente,aplicacion_mas_usada,tipo_evento,device_os_version
257007,41863526108385,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,Sin Wifi,,0.045441,True,88.0,1.0,210.0,4.017674e+18,0.0
285766,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,Sin Definir,,0.000000,False,0.0,0.0,0.0,0.000000e+00,0.0
70944,161514654074162,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,8.0,23.0,121.0,6.081165e+18,0.0
316713,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,0.0,0.0,0.0,0.000000e+00,0.0
155213,186034136943920,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,Sin Definir,list view,0.076904,False,13.0,1.0,210.0,4.017674e+18,0.0


In [57]:
features_por_ventana[0].shape

(385959, 23)

# Guardo los dataframes de features y labels

In [58]:
for i in range(0,7):
    labels[i].to_csv('labels_v'+str(i+1)+'.csv')
    features_por_ventana[i].to_csv('features_v'+str(i+1)+'.csv')  